In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import tensorflow as tf
import keras.models
#from keras.models import Sequential
from keras.layers import Conv2D , MaxPool2D ,Flatten , Dense, Dropout
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from PIL import Image
from keras.models import Model, Sequential
from keras.layers import GlobalAveragePooling2D , BatchNormalization;
#from models import custom_convnet
from keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
import warnings 
warnings.filterwarnings('ignore')

## Fake_Jobs_Posting

In [3]:
job=pd.read_csv(r"C:\Users\ADMIN\OneDrive\Documents\Jupyter_Data\fake_job_postings.csv")

In [4]:
job.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [5]:
job.isnull().sum()[job.isnull().sum()>0]

location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2696
benefits                7212
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
dtype: int64

In [6]:
job.fraudulent.value_counts()

fraudulent
0    17014
1      866
Name: count, dtype: int64

In [8]:
job['x'] = job['salary_range'].apply(str)+' '+job['company_profile'].apply(str)+' '+job['has_company_logo'].apply(str)+' '+job['location'].apply(str)+' '+job['benefits'].apply(str)

In [9]:
job=job.loc[:,['x','fraudulent']]

In [10]:
job.head()

,x,fraudulent
0,"nan We're Food52, and we've created a groundbr...",0
1,"nan 90 Seconds, the worlds Cloud Video Product...",0
2,nan Valor Services provides Workforce Solution...,0
3,nan Our passion for improving quality of life ...,0
4,nan SpotSource Solutions LLC is a Global Human...,0


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# x_train,x_test,y_train,y_test=train_test_split(job['X'],job['fraudulent'],test_size=0.2)

In [13]:
train,test=train_test_split(job)

In [15]:
df1=train[train.fraudulent==1]
train=pd.concat([train,df1,df1,df1]) #Oversampling

In [16]:
x_train = train.loc[:, "x" ]
x_test = test.loc[:, "x" ]
y_train = train.loc[:, "fraudulent" ]
y_test = test.loc[:, "fraudulent" ]

In [17]:
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [18]:
max_num_words=10000 
seq_len=50  
embedding_size=100

In [19]:
from keras.preprocessing.text import Tokenizer # to assign  number to words
from keras.preprocessing.sequence import pad_sequences # like padding in cnn add zeroes to make length of
# data uniform in starting 
tokenizer = Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(job['x']) 
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train=pad_sequences(x_train,maxlen=seq_len)
x_test=pad_sequences(x_test,maxlen=seq_len)
model=Sequential()
model.add(Embedding(input_dim=max_num_words,
         input_length=seq_len,
         output_dim=embedding_size))

In [21]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
model.add(LSTM(32))
model.add(Dense(2,activation='softmax'))
from tensorflow.keras.optimizers import Adam
adam=Adam(learning_rate=0.001)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
model.fit(x_train,y_train,epochs=10,batch_size=32,validation_split=0.2)

Epoch 1/10


582/582 [==============================] - 19s 28ms/step - loss: 0.1708 - accuracy: 0.9264 - val_loss: 0.1883 - val_accuracy: 0.9130
Epoch 2/10
582/582 [==============================] - 16s 27ms/step - loss: 0.0662 - accuracy: 0.9740 - val_loss: 0.0536 - val_accuracy: 0.9936
Epoch 3/10
582/582 [==============================] - 16s 27ms/step - loss: 0.0603 - accuracy: 0.9772 - val_loss: 0.0836 - val_accuracy: 0.9764
Epoch 4/10
582/582 [==============================] - 16s 27ms/step - loss: 0.0539 - accuracy: 0.9796 - val_loss: 0.0687 - val_accuracy: 0.9858
Epoch 5/10
582/582 [==============================] - 16s 28ms/step - loss: 0.0471 - accuracy: 0.9819 - val_loss: 0.0704 - val_accuracy: 0.9779
Epoch 6/10
582/582 [==============================] - 16s 27ms/step - loss: 0.0443 - accuracy: 0.9825 - val_loss: 0.0445 - val_accuracy: 0.9953
Epoch 7/10
582/582 [==============================] - 16s 28ms/step - loss: 0.0484 - accuracy: 0.9814 - val_loss: 0.0366 - val_accurac

In [391]:
# pred = (model.predict(x_test) > 0.4).astype(int)

In [23]:
pred=model.predict(x_test)
pred

140/140 [==============================] - 1s 5ms/step


array([[9.9999785e-01, 2.1382350e-06],
       [9.9999940e-01, 5.9906500e-07],
       [9.9999821e-01, 1.8132318e-06],
       ...,
       [9.9999690e-01, 3.0856017e-06],
       [1.6595942e-01, 8.3404052e-01],
       [9.9999940e-01, 5.6136611e-07]], dtype=float32)

In [24]:
pred_classes=pred.argmax(axis=1)

In [25]:
y_test=y_test.argmax(axis=1)

In [26]:
confusion_matrix(y_test,pred_classes)

array([[4158,  104],
       [  61,  147]], dtype=int64)

In [30]:
from sklearn.metrics import accuracy_score , classification_report

In [31]:
accuracy_score(y_test,pred_classes)

0.9630872483221476

In [29]:
print(classification_report(y_test,pred_classes))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      4262
           1       0.59      0.71      0.64       208

    accuracy                           0.96      4470
   macro avg       0.79      0.84      0.81      4470
weighted avg       0.97      0.96      0.96      4470



### Fake jobs using bidirectional LSTM

In [37]:
job=pd.read_csv(r"C:\Users\ADMIN\OneDrive\Documents\Jupyter_Data\fake_job_postings.csv")

In [38]:
job['x'] = job['salary_range'].apply(str)+' '+job['company_profile'].apply(str)+' '+job['has_company_logo'].apply(str)+' '+job['location'].apply(str)+' '+job['benefits'].apply(str)

In [39]:
job=job.loc[:,['x','fraudulent']]

In [40]:
train,test=train_test_split(job)

In [41]:
df1=train[train.fraudulent==1]
train=pd.concat([train,df1,df1,df1])

In [42]:
x_train = train.loc[:, "x" ]
x_test = test.loc[:, "x" ]
y_train = train.loc[:, "fraudulent" ]
y_test = test.loc[:, "fraudulent" ]

In [43]:
# x_train,x_test,y_train,y_test=train_test_split(job['X'],job['fraudulent'],test_size=0.2)

In [44]:
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [45]:
max_num_words=10000 
seq_len=50  
embedding_size=100

In [46]:
from keras.preprocessing.text import Tokenizer # to assign  number to words
from keras.preprocessing.sequence import pad_sequences # like padding in cnn add zeroes to make length of
# data uniform in starting 
tokenizer = Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(job['x']) 
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)


In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train=pad_sequences(x_train,maxlen=seq_len)
x_test=pad_sequences(x_test,maxlen=seq_len)
model=Sequential()
model.add(Embedding(input_dim=max_num_words,
         input_length=seq_len,
         output_dim=embedding_size))

In [48]:
from keras.layers import Bidirectional


In [49]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
model.add(Bidirectional(LSTM(32)))
model.add(Dense(2,activation='softmax'))
from tensorflow.keras.optimizers import Adam
adam=Adam(learning_rate=0.001)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [50]:
model.fit(x_train,y_train,epochs=8,batch_size=32,validation_split=0.2)

Epoch 1/8
384/384 [==============================] - 17s 36ms/step - loss: 0.1385 - accuracy: 0.9611 - val_loss: 0.6392 - val_accuracy: 0.6915
Epoch 2/8
384/384 [==============================] - 13s 33ms/step - loss: 0.0578 - accuracy: 0.9783 - val_loss: 0.3061 - val_accuracy: 0.8655
Epoch 3/8
384/384 [==============================] - 13s 34ms/step - loss: 0.0395 - accuracy: 0.9835 - val_loss: 0.3960 - val_accuracy: 0.8189
Epoch 4/8
384/384 [==============================] - 13s 34ms/step - loss: 0.0318 - accuracy: 0.9870 - val_loss: 0.2568 - val_accuracy: 0.8879
Epoch 5/8
384/384 [==============================] - 13s 35ms/step - loss: 0.0289 - accuracy: 0.9865 - val_loss: 0.2224 - val_accuracy: 0.9176
Epoch 6/8
384/384 [==============================] - 14s 35ms/step - loss: 0.0262 - accuracy: 0.9879 - val_loss: 0.3028 - val_accuracy: 0.8883
Epoch 7/8
384/384 [==============================] - 14s 36ms/step - loss: 0.0266 - accuracy: 0.9884 - val_loss: 0.2321 - val_accuracy: 0.9068

In [51]:
pred=model.predict(x_test)
pred

140/140 [==============================] - 2s 6ms/step


array([[9.9997747e-01, 2.2572813e-05],
       [9.9995244e-01, 4.7557947e-05],
       [9.9926633e-01, 7.3363195e-04],
       ...,
       [9.9947721e-01, 5.2284618e-04],
       [9.9996233e-01, 3.7723479e-05],
       [9.9999642e-01, 3.6075355e-06]], dtype=float32)

In [52]:
pred_classes=pred.argmax(axis=1)

In [53]:
y_test=y_test.argmax(axis=1)

In [54]:
confusion_matrix(y_test,pred_classes)

array([[4171,   79],
       [  67,  153]], dtype=int64)

In [55]:
print(classification_report(y_test,pred_classes))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      4250
           1       0.66      0.70      0.68       220

    accuracy                           0.97      4470
   macro avg       0.82      0.84      0.83      4470
weighted avg       0.97      0.97      0.97      4470

